In [1]:
# created on Jan 28, 2020
# @author:          Kyle Chen
# @email:           kyle00@uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

!python -m pip install tweepy
import tweepy, json, time

In [2]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.f = open(file, 'a', encoding="utf-8")
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                # {'limit': {'track': 13, 'timestamp_ms': '1585851016736'}}

                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    record = '%s, %s, %f, %f, %s \n' % (username, created_at, lng, lat, text)
                    print(record)
                    self.f.write(record)
                else:
                    pass
        else:
            self.f.close()
            print("finished")
            return False

In [3]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = "assets/tweets.csv"

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "nV8FL4HNVTEcFzW5n1cYTzSax"
    consumer_secret = "G4smWCa3foP4qmN6gqUlDXVimrprzWhxillZhYTgSlQQouUtJW"
    access_token = "1354397815468789760-U06fKuX1kcXvD6OYFRjxNfSiEqYdqj"
    access_token_secret = "sdQrrL06rQr9ACqlZDHOM4di32WkMus2UjG9gPWVCexqU"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    LOCATIONS = [-124.7771694, 24.520833, -66.947028, 49.384472,  # Contiguous US
                 -164.639405, 58.806859, -144.152365, 71.76871,  # Alaska
                 -160.161542, 18.776344, -154.641396, 22.878623]  # Hawaii

    stream_listener = StreamListener(time_limit=60, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(locations=LOCATIONS, track=['Kobe'], is_async=True)

{'created_at': 'Thu Jan 28 08:07:44 +0000 2021', 'id': 1354702688651538436, 'id_str': '1354702688651538436', 'text': '@Kassmysass_ Soo trueee omg', 'display_text_range': [13, 27], 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': 1354701900223045634, 'in_reply_to_status_id_str': '1354701900223045634', 'in_reply_to_user_id': 25425552, 'in_reply_to_user_id_str': '25425552', 'in_reply_to_screen_name': 'Kassmysass_', 'user': {'id': 2920252328, 'id_str': '2920252328', 'name': 'Eddy V = Lit Nurse 🏨💉 = EDM Kronk 😈👨🏻\u200d🍳😇', 'screen_name': 'colombian085', 'location': 'miami florida ', 'url': 'https://www.instagram.com/eddyv305/', 'description': 'snapchat: colombian03. Cali, Colombia 🇨🇴. Trance, House, Techno, Q Crew🥺❤️:🎵👽 @H2hoes13 P.L.U.R .✨🙌💥🔊B.L.M🖤Poly🌈🦄🖤💛💙❤️ @LifeisVOYD 👼🏼🙏🏼\U0001fa78🕳', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 3404, 'friends_count': 4996